<a href="https://colab.research.google.com/github/aymanish/INT2-ImageNet-Pytorch-Model/blob/main/INT2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
In order to use torchVision you must run the command
'tensorboard --logdir=runs'
From terminal in the same directory as the python file

Then open http://localhost:6006

Install PyTorch at https://pytorch.org/get-started/locally/
"""
# using the time you can tell the program to run for an amount of time instead of a specific number of epochs
from timeit import default_timer as timer
from datetime import datetime
from pathlib import Path

# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:

#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# obtains the official training and test sets for the CIFAR10 Dataset
#trainset = torchvision.datasets.CIFAR10(
#    root='./data',
#    train=True,
#    download=True,
#    transform=transform
#)
#testset = torchvision.datasets.CIFAR10(
#    root='./data',
#    train=False,
#    download=True,
#    transform=transform
#)

In [ ]:
#stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#RANDOM IMAGE AUGMENTATION TO THE TRAINING IMAGES

random_transform = transforms.Compose([transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# obtains the official training and test sets for the CIFAR10 Dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=random_transform
)
testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

100%|██████████| 170498071/170498071 [00:04<00:00, 35486229.93it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#how many values are tested in each iteration
batch_size = 50

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1) #numworkers = 1
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for images, lables in testloader:
  print(f"Shape of X [N, C, H, W] : {images.shape}")
  print(f"Shape of y: {lables.shape} {lables.dtype}")
  break

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Shape of X [N, C, H, W] : torch.Size([50, 3, 32, 32])
Shape of y: torch.Size([50]) torch.int64


In [ ]:
#show images
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(torchvision.utils.make_grid(images[:64], nrow=8).permute(1, 2, 0))
        break

In [ ]:
show_batch(trainloader)

In [ ]:
'''
# Get training device. This will be the CPU or GPU(cuda) depending on availability
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

# Define model. In pytorch this inherits from nn.Module. The networks layers are defined in __init__.
# Forward is how we specify how data passes through the network. If the GPU is available we pass operations to that as it is faster.
class NeuralNetwork(nn.Module):
     def __init__(self, num_classes=len(classes)):
         super(NeuralNetwork, self).__init__()
         self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),#change strides on some convolutions first as 1 and increase kernel size
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )

#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5,120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

         self.avgpool = nn.AdaptiveAvgPool2d((6,6))

         self.linear = nn.Sequential(
            nn.Dropout(),
            # if you set stride to equal 2 then change the fisrt 4096 to 255 * 2 * 2
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

     def forward(self, x):
        x = self.conv(x)
        # if you set stride to equal 2 then change 4096 to 255 * 2 * 2
        x = x.view(x.size(0), 4096)
        x = torch.flatten(x, 1)
        x = self.linear(x)
        return x


model = NeuralNetwork().to(device)
'''

'\n# Get training device. This will be the CPU or GPU(cuda) depending on availability\ndevice = torch.device(\'cuda:0\' if torch.cuda.is_available() else \'cpu\')\nprint(f"Using {device} device")\n\n# Define model. In pytorch this inherits from nn.Module. The networks layers are defined in __init__. \n# Forward is how we specify how data passes through the network. If the GPU is available we pass operations to that as it is faster.\nclass NeuralNetwork(nn.Module):\n     def __init__(self, num_classes=len(classes)):\n         super(NeuralNetwork, self).__init__()\n         self.conv = nn.Sequential(\n            nn.Conv2d(3, 64, kernel_size=3, padding=1),#change strides on some convolutions first as 1 and increase kernel size\n            nn.ReLU(inplace=True),\n            nn.MaxPool2d(kernel_size=2),\n            nn.Conv2d(64, 192, kernel_size=3, padding=1),\n            nn.ReLU(inplace=True),\n            nn.MaxPool2d(kernel_size=2),\n            nn.Conv2d(192, 384, kernel_size=3, pa

In [ ]:
# Get training device. This will be the CPU or GPU(cuda) depending on availability
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

# Define model. In pytorch this inherits from nn.Module. The networks layers are defined in __init__.
# Forward is how we specify how data passes through the network. If the GPU is available we pass operations to that as it is faster.
class NeuralNetwork(nn.Module):
     def __init__(self, num_classes=len(classes)):
         super(NeuralNetwork, self).__init__()
         self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),#change strides on some convolutions first as 1 and increase kernel size
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )

#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5,120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

         self.avgpool = nn.AdaptiveAvgPool2d((6,6))

         self.linear = nn.Sequential(
            nn.Dropout(),
            # if you set stride to equal 2 then change the fisrt 4096 to 255 * 2 * 2
            nn.Linear(4096*2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

     def forward(self, x):
        x = self.conv(x)
        # if you set stride to equal 2 then change 4096 to 255 * 2 * 2
        x = x.view(x.size(0), 4096*2)
        x = torch.flatten(x, 1)
        x = self.linear(x)
        return x


model = NeuralNetwork().to(device)

Using cuda:0 device


In [ ]:
loss_fn = nn.CrossEntropyLoss()
lr = 0.001
optimizerSGD = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
optimizerAdam = torch.optim.Adam(model.parameters(), lr = lr)
optimizerRMSprop = torch.optim.RMSprop(model.parameters(), lr =lr)
optimizerAdaDelta = torch.optim.Adadelta(model.parameters(), lr =lr)
optimizerAdaGrad = torch.optim.Adagrad(model.parameters(),lr = lr)


In [ ]:
def train(trainloader, model, loss_fn, optimizer):
    size = len(trainloader.dataset)
    model.train()
    running_loss = 0.0
    batch_loss = 0.0
    batchTime = timer()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:
            loss, current = loss.item(), i * len(inputs)
            print(f'[{(i + 1) * batch_size:>5d}/{size:5d}] loss: {(running_loss - batch_loss) / 100:.4f} (batch time: {timer() - batchTime:>.4f})')
            batchTime = timer()
            batch_loss = running_loss

    return running_loss

In [ ]:
def test(testloader, model, loss_fn):

    size = len(testloader.dataset)
    num_batches = len(testloader)
    model.eval()
    test_loss, correct = 0, 0

    class_probs = []
    class_label = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            output = model(images)

            test_loss += loss_fn(output, labels).item()
            correct += (output.argmax(1) == labels).type(torch.float).sum().item()

            class_probs_batch = [F.softmax(el, dim=0) for el in output]

            class_probs.append(class_probs_batch)
            class_label.append(labels)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

    test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
    test_label = torch.cat(class_label)

In [ ]:
dt_string = datetime.now().strftime("%Y%m%d_%H;%M;%S")
Path(f"./models/{dt_string}/").mkdir(parents=True, exist_ok=True)

writer = SummaryWriter(f'./logs/{dt_string}')
writer.add_graph(model, images.to(device))

In [ ]:
# Set t really large if running for a specific number of times (endTime is in seconds)
startTime = timer()
endTime = 72000 # One hour = 3600
lastEpochTime = startTime
# Set e really high if running based on time
e = 100000000000
for epoch in range(e):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizerSGD)
    #testing only on certain epochs
    if epoch % 10 == 0:
      test(testloader, model, loss_fn)

    print(f"Time elapsed: {round(timer() - lastEpochTime, 4)} seconds\n")
    lastEpochTime = timer()
    #PATH = f'./models/{dt_string}/epoch_{epoch + 1}.pth'
    #torch.save(model.state_dict(), PATH)
    if timer() - startTime >= endTime:
        break
print("Done!")

Epoch 1
-------------------------------
[25000/50000] loss: 11.5121 (batch time: 22.0893)
[50000/50000] loss: 11.5108 (batch time: 20.5440)
Test Error: 
Accuracy: 15.6%, Avg loss: 2.301537
Time elapsed: 47.5297 seconds

Epoch 2
-------------------------------
[25000/50000] loss: 11.5068 (batch time: 21.0538)
[50000/50000] loss: 11.4987 (batch time: 22.1443)
Time elapsed: 43.2217 seconds

Epoch 3
-------------------------------
[25000/50000] loss: 11.4626 (batch time: 21.1682)
[50000/50000] loss: 11.0841 (batch time: 21.9559)
Time elapsed: 43.1468 seconds

Epoch 4
-------------------------------
[25000/50000] loss: 10.0389 (batch time: 21.8672)
[50000/50000] loss: 9.3411 (batch time: 21.3018)
Time elapsed: 43.1925 seconds

Epoch 5
-------------------------------
[25000/50000] loss: 8.9438 (batch time: 22.4353)
[50000/50000] loss: 8.7122 (batch time: 21.2279)
Time elapsed: 43.6879 seconds

Epoch 6
-------------------------------
[25000/50000] loss: 8.5216 (batch time: 22.4539)
[50000/500

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        # imgshow(make_grid(images)) # outputs all test images
        output = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

class_probs = []
class_label = []
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        output = model(images)

        _, predictions = torch.max(output, 1)

        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

        class_probs_batch = [F.softmax(el, dim=0) for el in output]

        class_probs.append(class_probs_batch)
        class_label.append(labels)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_label = torch.cat(class_label)

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy of the network on the test images: 89 %
Accuracy for class: plane is 90.2 %
Accuracy for class: car   is 96.1 %
Accuracy for class: bird  is 89.4 %
Accuracy for class: cat   is 79.2 %
Accuracy for class: deer  is 88.5 %
Accuracy for class: dog   is 82.8 %
Accuracy for class: frog  is 91.1 %
Accuracy for class: horse is 91.9 %
Accuracy for class: ship  is 92.9 %
Accuracy for class: truck is 93.3 %


In [ ]:
# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_label, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_label)